In [1]:
import os
import zipfile
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error

import matplotlib.pyplot as plt

In [2]:
def unzip_dataset(
    source: str,
    destination: str,
    remove_source: bool = False
    ) -> Path:

    data_dir_path = Path("data")
    data_path = data_dir_path / destination
    source_path = Path(source)

    if not data_dir_path.is_dir():
      data_dir_path.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(source_path, "r") as zip_ref:
      zip_ref.extractall(data_path)

    if remove_source:
      os.remove(source)

    return data_path

unzip_dataset(
    source="air+quality.zip",
    destination="air_quality"
)

PosixPath('data/air_quality')

In [3]:
original_df = pd.read_excel("data/air_quality/AirQualityUCI.xlsx")
original_df.shape

(9357, 15)

In [4]:
def transform_time(time: int) -> str:
  if time > 5 and time <=12:
    return 'morning'
  elif time > 12 and time <= 17:
    return 'afternoon'
  elif time > 17 and time <= 21:
    return 'evening'
  else:
    return 'night'

def transform_date(date: int) -> str:
  if date > 3 and date <=5:
    return 'spring'
  elif date > 5 and date <= 8:
    return 'summer'
  elif date > 8 and date <= 11:
    return 'autumn'
  else:
    return 'winter'

In [5]:
def split_stratified(dataset):
  dataset["Season"] = dataset["Date"].apply(lambda date: date.month)
  dataset["Season"] = dataset["Season"].apply(transform_date)

  print(dataset.shape)

  train, test = train_test_split(dataset, test_size=0.2, stratify=dataset["Season"], random_state=42)

  for split_ in (train, test):
    split_.drop("Season", axis=1, inplace=True)
  return train, test

changed_df = original_df.copy()
train_data, test_data = split_stratified(changed_df)
X_train, y_train = train_data.drop('T', axis=1), train_data['T']
X_test, y_test = test_data.drop('T', axis=1), test_data['T']

(9357, 16)


In [6]:
class DataTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass

  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    X['Date'] = X['Date'].apply(lambda date: date.month)
    X['Time'] = X['Time'].apply(lambda time: time.hour)
    X['Time'] = X['Time'].apply(transform_time)
    X['Date'] = X['Date'].apply(transform_date)
    return X

In [7]:
X_test.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH
811,2004-04-13,13:00:00,1.5,1009.25,141,7.658833,885.00,112.0,993.75,87.0,1438.50,750.75,38.150000,0.700562
1907,2004-05-29,05:00:00,-200.0,737.75,-200,1.501200,544.75,18.0,1562.00,29.0,1265.75,449.50,49.749999,0.996961
2998,2004-07-13,16:00:00,1.7,905.00,-200,8.775651,930.50,124.0,875.50,90.0,1460.25,775.75,21.300000,0.805444
4304,2004-09-06,02:00:00,0.6,854.00,-200,1.706070,561.50,-200.0,1105.50,-200.0,1224.25,400.00,42.950000,1.290619
6001,2004-11-15,19:00:00,3.8,1029.00,-200,16.594653,1197.75,446.0,693.50,175.0,1305.00,1168.50,30.200000,0.438446


In [13]:
numerical_columns = X_train.select_dtypes(include=np.number).columns.to_list()
categorical_columns = X_train.select_dtypes(exclude=np.number).columns.to_list()
categorical_columns, numerical_columns

(['Date', 'Time'],
 ['CO(GT)',
  'PT08.S1(CO)',
  'NMHC(GT)',
  'C6H6(GT)',
  'PT08.S2(NMHC)',
  'NOx(GT)',
  'PT08.S3(NOx)',
  'NO2(GT)',
  'PT08.S4(NO2)',
  'PT08.S5(O3)',
  'RH',
  'AH'])

In [14]:
numerical_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy="mean", missing_values=-200)),
        ('scaler', StandardScaler())
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ('transformer', DataTransformer()),
        ('ohe_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))
    ]
)

full_pipeline = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, numerical_columns),
        ('categorical', categorical_pipeline, categorical_columns)
    ]
)

In [18]:
linear_model = Pipeline(
    steps=[
        ('preprocessor', full_pipeline),
        ('model', LinearRegression())
    ]
)

sgd_model = Pipeline(
    steps=[
        ('preprocessor', full_pipeline),
        ('model', SGDRegressor())
    ]
)

tree_model = Pipeline(
    steps=[
        ('preprocessor', full_pipeline),
        ('model', DecisionTreeRegressor())
    ]
)

rdf_model = Pipeline(
    steps=[
        ('preprocessor', full_pipeline),
        ('model', RandomForestRegressor())
    ]
)

svm_model = Pipeline(
    steps=[
        ('preprocessor', full_pipeline),
        ('model', SVR())
    ]
)

models = [tree_model, rdf_model, sgd_model, svm_model]

In [15]:
def train_model(model, X_train, y_train, X_test, y_test):
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  metrics = root_mean_squared_error(y_test, predictions)
  return metrics

def test_model_cv(model, X_test, y_test):
  scores = -cross_val_score(model, X_test, y_test, scoring="neg_root_mean_squared_error", cv=10)
  print(f"Model: {model}\n")
  print(f"Scores: {scores}\n")
  print(f"Mean: {scores.mean()}\n")
  print(f"Standard Deviation: {scores.std()}\n")
  return scores

def grid_search_model(model, param_grid, X, y):

  print(f'{model}\n')
  grid_search = GridSearchCV(
      model,
      param_grid,
      cv=5,
      scoring='neg_root_mean_squared_error',
      verbose=1)
  grid_search.fit(X, y)
  best_model = grid_search.best_estimator_
  return best_model

In [17]:
sgd_regressor_grid = {
    'model__loss': ['squared_error', 'huber'],
    'model__penalty': ['l2', 'l1'],
    'model__alpha': [0.00001, 0.0001, 0.001]
}


svm_regressor_grid = {
    'model__kernel': ['linear', 'rbf'],
    'model__C': [0.1, 1, 10],
    'model__gamma': ['scale', 'auto']
}

tree_regressor_grid = {
    'model__criterion': ['squared_error', 'absolute_error'],
    'model__splitter': ['best', 'random'],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

rdf_regressor_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

param_grids = [tree_regressor_grid, rdf_regressor_grid, sgd_regressor_grid, svm_regressor_grid]

In [19]:
best_models = []
best_models.append(linear_model)
for model, grid in zip(models, param_grids):
  %time best_models.append(grid_search_model(model, grid, X_train, y_train))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(missing_values=-200)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH', 'AH']),
                                                 ('categorical',
                                     

In [20]:
for model in best_models:
  %time test_model_cv(model, X_test, y_test)
  print("-" * 50)

Model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(missing_values=-200)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH', 'AH']),
                                                 ('categorical',
                              

In [21]:
model_names = ['Linear_model', 'DecisionTree_model', 'RandomForest_model', 'SGD_model', 'SVM_model']
models_path = Path('models')

models_path.mkdir(parents=True, exist_ok=True)

for model, model_name in zip(best_models, model_names):
  with open(models_path / f'{model_name}.pkl', 'wb') as file:
    pickle.dump(model, file)

In [22]:
rdf_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(missing_values=-200)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH', 'AH']),
                                                 ('categorical',
                                                  Pipeline(steps=[('transformer',
                                                                   DataTransformer()),
                                                                  ('ohe_encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Date', 'Time'])])),
                ('model', RandomForestRegressor())])